Simple MA crossover strategy to test backtesting.py package

In [11]:

from backtesting.test import GOOG
GOOG.head()

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900


In [12]:
import sys
print(sys.executable)

/opt/anaconda3/bin/python


In [13]:
import pandas as pd


def SMA(values, n):
    return pd.Series(values).rolling(n).mean()

New strategy needs to extend Strategy class and override two abstract methods init() and next()

init alr invoked before strategy is run. 
next is iteratievley called by the backtest instance foe each data frame row 
backtesting.py can't trade within candle sticks, for intraday need to use hourly data rip

In [14]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [5]:
from backtesting import Backtest

bt = Backtest(GOOG, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Backtest.run:   0%|          | 0/2128 [00:00<?, ?bar/s]

/var/folders/f1/v4_lp9ks0dj4__tzb7_dw4k00000gn/T/ipykernel_78858/134679513.py:4: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()


Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    94.27374
Equity Final [$]                  56263.51934
Equity Peak [$]                   56309.05934
Commissions [$]                   10563.95154
Return [%]                          462.63519
Buy & Hold Return [%]               607.37036
Return (Ann.) [%]                    22.46598
Volatility (Ann.) [%]                 37.4129
CAGR [%]                             14.99343
Sharpe Ratio                          0.60049
Sortino Ratio                          1.1445
Calmar Ratio                           0.6621
Alpha [%]                           450.62135
Beta                                  0.01978
Max. Drawdown [%]                   -33.93159
Avg. Drawdown [%]                    -6.16072
Max. Drawdown Duration      830 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                          

In [15]:
bt.plot()

/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:709: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:709: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2221', ...)

In [16]:
stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

/opt/anaconda3/lib/python3.12/site-packages/backtesting/backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/50 [00:00<?, ?it/s]

Backtest.run:   0%|          | 0/2138 [00:00<?, ?bar/s]

/opt/anaconda3/lib/python3.12/site-packages/backtesting/backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)


Backtest.run:   0%|          | 0/2108 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/2083 [00:00<?, ?bar/s]

/opt/anaconda3/lib/python3.12/site-packages/backtesting/backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/multiprocessing/resource_tracker.py", line 239, in main
    cache[rtype].remove(name)
KeyError: '/psm_b8a3afa5'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/multiprocessing/resource_tracker.py", line 239, in main
    cache[rtype].remove(name)
KeyError: '/psm_22bf3d0f'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/multiprocessing/resource_tracker.py", line 239, in main
    cache[rtype].remove(name)
KeyError: '/psm_f6516112'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/multiprocessing/resource_tracker.py", line 239, in main
    cache[rtype].remove(name)
KeyError: '/psm_b29a1a60'
Traceb

Backtest.run:   0%|          | 0/2133 [00:00<?, ?bar/s]

/opt/anaconda3/lib/python3.12/site-packages/backtesting/backtesting.py:1545: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = self.run(**dict(zip(heatmap.index.names, best_params)))


Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                     98.1378
Equity Final [$]                  77829.05462
Equity Peak [$]                    84982.1921
Commissions [$]                   30771.03746
Return [%]                          678.29055
Buy & Hold Return [%]               687.98749
Return (Ann.) [%]                    27.21765
Volatility (Ann.) [%]                43.21089
CAGR [%]                             18.05088
Sharpe Ratio                          0.62988
Sortino Ratio                         1.27855
Calmar Ratio                          0.61089
Alpha [%]                           614.80095
Beta                                  0.09228
Max. Drawdown [%]                   -44.55387
Avg. Drawdown [%]                    -5.81107
Max. Drawdown Duration     1558 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                          

In [17]:
stats._strategy


<Strategy SmaCross(n1=10,n2=15)>

In [18]:
bt.plot(plot_volume=False, plot_pl=False)


/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:709: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:709: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2484', ...)